### 19.4.2 Real Data, Real Practice - Deep Learning Classification Model (Tabular input data)


Data questions to consider:
* What about this dataset makes it complex? Is it a variable? Is it the distribution of values? Is it the size of the dataset?


* Which variables should I investigate prior to implementing my model? What does the distribution look like? Hint: Use Pandas’ Series.plot.density() method to find out.


* What outcome am I looking for from the model? Which activation function should I use to get my desired outcome?


* What is my accuracy cutoff? In other words, what percent testing accuracy must my model exceed?

Dataset : Contains employee career profile and attrition (departure from company) information, such as age, department, job role, work/life balance, and so forth. Using this data, we can generate a deep learning model that can help to identify whether or not a person is likely to depart from the company given his or her current employee profile.

### Import Dependencies 

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### 19.4.3 Preprocessing: Encode all categorical data

In [3]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()


In [4]:
# Check the number of unique values in each column to bucket
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


### Training and Testing Datasets
Input values --> separate into features (X)
output target value --> separate into target (y)

In [7]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

### Standardization of Data

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Check data and dataframe
attrition_df.head(10)

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
5,32,1005,2,2,1,8,4,79,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
6,59,1324,3,3,1,10,3,81,4,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
7,30,1358,24,1,1,11,4,67,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
8,38,216,23,3,1,12,4,44,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
9,36,1299,27,3,1,13,3,94,3,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


### 19.4.4 Deep Learning Model Design

Notes:
For our input layer, we must add the number of input features equal to the number of variables in our feature DataFrame.
In our hidden layers, our deep learning model structure will be slightly different—we’ll add two hidden layers with only a few neurons in each layer. To create the second hidden layer, we’ll add another Keras Dense class while defining our model. All of our hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
In the output layer, we’ll use the same parameters from our basic neural network including the sigmoid activation function. The sigmoid activation function will help us predict the probability that an employee is at risk for attrition.


In [10]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#### Skip code below and go to cells at bottom for checkpoint assignment

In [ ]:
# Compile the model and define the loss and accuracy metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### 19.4.5 Train and Evaluate the Model

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Summary: the model was able to correctly identify employees are at the risk of attrition approximately 87% of the time. The deep learning model is a decent reliable classifier since more than 30 different variables were used for input data, more than 1,400 data points.

### 19.6.1 Checkpoints Are Not Just for Video Games

### How to store a trained model to be access outside training environment

With TensorFlow, we have the ability to save and load neural network models at any stage, including partially trained models. When building a TensorFlow model, if we use Keras’ `ModelCheckpoint` method, we can save the model weights after it tests a set number of data points. Then, at any point, we can reload the checkpoint weights and resume model training.

### Outline of Notebook Code:

1. Import dependencies.

2. Import the input dataset.

3. Generate categorical variable list.

4. Create a OneHotEncoder instance.

5. Fit and transform the OneHotEncoder.

6. Add the encoded variable names to the DataFrame.

7. Merge one-hot encoded features and drop the originals.

8. Split the preprocessed data into features and target arrays.

9. Split the preprocessed data into training and testing dataset.

10. Create a StandardScaler instance.

11. Fit the StandardScaler.

12. Scale the data.

13. Define the model.

14. Add first and second hidden layers.

15. Add the output layer.

16. Check the structure of the model.

### Checkpoint version of compiling the previous deep neural model

In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

### Train the model

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
  32/1102 [..............................] - ETA: 4s - loss: 0.7426 - acc: 0.5312
Epoch 00001: saving model to checkpoints/weights.01.hdf5
1102/1102 [==============================] - 0s 277us/sample - loss: 0.6955 - acc: 0.5808
Epoch 2/100
 800/1102 [====================>.........] - ETA: 0s - loss: 0.6130 - acc: 0.7462
Epoch 00002: saving model to checkpoints/weights.02.hdf5
1102/1102 [==============================] - 0s 96us/sample - loss: 0.6077 - acc: 0.7541
Epoch 3/100
  32/1102 [..............................] - ETA: 0s - loss: 0.5083 - acc: 0.8438
Epoch 00003: saving model to checkpoints/weights.03.hdf5
1102/1102 [==============================] - 0s 79us/sample - loss: 0.5572 - acc: 0.8113
Epoch 4/100
  32/1102 [..............................] - ETA: 0s - loss: 0.4748 - acc: 0.9062
Epoch 00004: saving model to checkpoints/weights.04.hdf5
1102/1102 [==============================] - 0s 72us/sample - loss: 0.5218 - acc: 0.8203
Epoch 5/100
  32/1102 [................

1102/1102 [==============================] - 0s 58us/sample - loss: 0.2735 - acc: 0.8884
Epoch 34/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2255 - acc: 0.8438
Epoch 00034: saving model to checkpoints/weights.34.hdf5
1102/1102 [==============================] - 0s 60us/sample - loss: 0.2694 - acc: 0.8929
Epoch 35/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2482 - acc: 0.8750
Epoch 00035: saving model to checkpoints/weights.35.hdf5
1102/1102 [==============================] - 0s 59us/sample - loss: 0.2664 - acc: 0.8929
Epoch 36/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1915 - acc: 0.9062
Epoch 00036: saving model to checkpoints/weights.36.hdf5
1102/1102 [==============================] - 0s 58us/sample - loss: 0.2617 - acc: 0.8984
Epoch 37/100
  32/1102 [..............................] - ETA: 0s - loss: 0.4551 - acc: 0.8125
Epoch 00037: saving model to checkpoints/weights.37.hdf5
1102/1102 [=========================

  32/1102 [..............................] - ETA: 0s - loss: 0.3628 - acc: 0.8750
Epoch 00066: saving model to checkpoints/weights.66.hdf5
1102/1102 [==============================] - 0s 60us/sample - loss: 0.2027 - acc: 0.9183
Epoch 67/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1106 - acc: 0.9688
Epoch 00067: saving model to checkpoints/weights.67.hdf5
1102/1102 [==============================] - 0s 76us/sample - loss: 0.2014 - acc: 0.9211
Epoch 68/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1689 - acc: 0.9375
Epoch 00068: saving model to checkpoints/weights.68.hdf5
1102/1102 [==============================] - 0s 72us/sample - loss: 0.2000 - acc: 0.9183
Epoch 69/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2116 - acc: 0.9062
Epoch 00069: saving model to checkpoints/weights.69.hdf5
1102/1102 [==============================] - 0s 84us/sample - loss: 0.1991 - acc: 0.9174
Epoch 70/100
  32/1102 [.........................

1102/1102 [==============================] - 0s 60us/sample - loss: 0.1660 - acc: 0.9383
Epoch 99/100
  32/1102 [..............................] - ETA: 0s - loss: 0.0664 - acc: 1.0000
Epoch 00099: saving model to checkpoints/weights.99.hdf5
1102/1102 [==============================] - 0s 67us/sample - loss: 0.1652 - acc: 0.9401
Epoch 100/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1721 - acc: 0.9375
Epoch 00100: saving model to checkpoints/weights.100.hdf5
1102/1102 [==============================] - 0s 59us/sample - loss: 0.1648 - acc: 0.9419


### Evalute the model

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.3518 - acc: 0.8750
Loss: 0.3517586044643236, Accuracy: 0.875


### Test and restore weights - use Keras Sequential model "load weights" for Keras model

In [16]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

### Evaluate the model

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.3520 - acc: 0.8777
Loss: 0.3520028124684873, Accuracy: 0.8777173757553101


### Summary
Loss and accuracy has been restored to output similar to stored values. 

### Saving the model after training

#### 1. Export saved model

In [18]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

#### 2. Import model to new object

In [19]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


#### 3. Evaluate the model using the test data

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


368/368 - 0s - loss: 0.3520 - acc: 0.8777
Loss: 0.3520028124684873, Accuracy: 0.8777173757553101


### Summary
Looking at the final results, our imported model was able to reproduce the exact same performance metrics as the original model. Using this same procedure, we can import any type of Keras model for evaluation on a dataset with the same features.